In [37]:
# Set some notebook defaults
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

# Display information about the environment
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

(v"1.5.3", 8, 8)

In [38]:
using Random, Distributions, BenchmarkTools, Tullio, LoopVectorization, DataFrames, StaticKernels, SantasLittleHelpers

In [39]:
N = 10^8
d = Normal()
df = DataFrame(:data => rand(d, N))

,data
,Float64
1,-0.897292
2,1.02837
3,-0.222221
4,0.221688
5,-0.89454
6,-1.61098
7,0.00556986
8,0.417488
9,0.334858


In [40]:
# Calculate sma using StaticKernels and SantasLittleHelpers
k = makekernel(mean,-9:0)
df[!, "sma_sk"] = applyrolling(k, df.data)
df

,data,sma_sk
,Float64,Float64
1,-0.897292,-0.897292
2,1.02837,0.0655403
3,-0.222221,-0.0303801
4,0.221688,0.0326368
5,-0.89454,-0.152798
6,-1.61098,-0.395829
7,0.00556986,-0.338486
8,0.417488,-0.24399
9,0.334858,-0.179673


In [41]:
tmp = Array{Union{Missing, Float64}}(missing, length(df.data));

function f_tullio(data, sma)
    out_view = @view sma[10:end]
    @tullio out_view[i] = 0.1*data[i+o]  o in 0:9  # verbose=true;
    sma
end

df[!, "sma_tullio"] = f_tullio(df.data, tmp)
df

,data,sma_sk,sma_tullio
,Float64,Float64,Float64?
1,-0.897292,-0.897292,missing
2,1.02837,0.0655403,missing
3,-0.222221,-0.0303801,missing
4,0.221688,0.0326368,missing
5,-0.89454,-0.152798,missing
6,-1.61098,-0.395829,missing
7,0.00556986,-0.338486,missing
8,0.417488,-0.24399,missing
9,0.334858,-0.179673,missing


In [42]:
out = Vector{Union{Missing, Float64}}(undef, N)

function rollingmean2(x, m, out)
    N = length(x)
    out[1:(m-1)] .= missing
    sm = mean(@view x[1:m])
    out[m] = sm
    @inbounds @simd for i in (m+1):length(x)
        sm += (x[i] - x[i-m]) / m
        out[i] = sm
    end
    return out 
end

rollingmean2 (generic function with 2 methods)

In [43]:
df[!, "sma_simd"] = rollingmean2(df.data, 10)
df

,data,sma_sk,sma_tullio,sma_simd
,Float64,Float64,Float64?,Float64?
1,-0.897292,-0.897292,missing,missing
2,1.02837,0.0655403,missing,missing
3,-0.222221,-0.0303801,missing,missing
4,0.221688,0.0326368,missing,missing
5,-0.89454,-0.152798,missing,missing
6,-1.61098,-0.395829,missing,missing
7,0.00556986,-0.338486,missing,missing
8,0.417488,-0.24399,missing,missing
9,0.334858,-0.179673,missing,missing


In [44]:
@benchmark applyrolling(k, df.data)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.133 s (0.09% GC)
  median time:      1.188 s (0.09% GC)
  mean time:        1.195 s (3.62% GC)
  maximum time:     1.260 s (10.65% GC)
  --------------
  samples:          5
  evals/sample:     1

In [45]:
@benchmark f_tullio(df.data, tmp)

BenchmarkTools.Trial: 
  memory estimate:  7.61 KiB
  allocs estimate:  115
  --------------
  minimum time:     134.007 ms (0.00% GC)
  median time:      155.243 ms (0.00% GC)
  mean time:        154.218 ms (0.00% GC)
  maximum time:     176.329 ms (0.00% GC)
  --------------
  samples:          33
  evals/sample:     1

In [46]:
@benchmark rollingmean2(df.data, 10, out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     130.821 ms (0.00% GC)
  median time:      148.269 ms (0.00% GC)
  mean time:        146.676 ms (0.00% GC)
  maximum time:     162.207 ms (0.00% GC)
  --------------
  samples:          35
  evals/sample:     1